In [2]:
%%writefile build_matrix_A.py
import numpy as np
from scipy.sparse import diags, kron, eye

def build_matrix_A(Nx, Ny):
    # Define the grid spacing
    hx = 1 / (Nx + 1)
    hy = 1 / (Ny + 1)
    
    # Create a 2D grid of coordinates
    x = np.linspace(0, 1, Nx + 2)
    y = np.linspace(0, 1, Ny + 2)
    X, Y = np.meshgrid(x, y)
    
    # Coefficients for the PDE
    e_xy = np.exp(X * Y)
    a = 20 * (X + Y)
    b = (X + Y)
    c = 1 / (1 + X + Y)
    
    # The five-point central difference stencil for the second derivative terms
    # Compute the coefficients for the central difference approximation
    diag_x = e_xy[1:-1, 1:-1].flatten()
    diag_y = e_xy[1:-1, 1:-1].T.flatten()
    
    # Build the 1D laplacians
    laplacian_x = diags([diag_x, -2 * diag_x, diag_x], [-1, 0, 1], shape=(Nx, Nx))
    laplacian_y = diags([diag_y, -2 * diag_y, diag_y], [-1, 0, 1], shape=(Ny, Ny))
    
    # Construct the full 2D Laplacian as a Kronecker sum
    A_2D = (kron(eye(Ny), laplacian_x) + kron(laplacian_y, eye(Nx))) / (hx * hy)
    
    # Construct the first derivative terms
    # Assuming a 5-point central difference for the first derivative (you'll need to adjust this if not)
    first_deriv_x = diags([-a[1:-1, 1:-1].flatten(), a[1:-1, 1:-1].flatten()], [-1, 1], shape=(Nx, Nx))
    first_deriv_y = diags([-b[1:-1, 1:-1].T.flatten(), b[1:-1, 1:-1].T.flatten()], [-1, 1], shape=(Ny, Ny))
    
    # Add the first derivative terms to the 2D Laplacian
    A_2D += (kron(eye(Ny), first_deriv_x) + kron(first_deriv_y, eye(Nx))) / 2
    
    # Add the zeroth-order term
    A_2D += diags(c[1:-1, 1:-1].flatten(), 0)
    
    return A_2D

Writing build_matrix_A.py
